In [2]:
import networkx as nx
from networkx.algorithms import community
from pyvis.network import Network
import math
import numpy as np

In [3]:
def cond1(n):
    for b in (0,1):
        for a in model_graph.adj[n]:
            if n in communities[b] and not(a in communities[b]):
                return True

    return False

In [4]:
def cond2(n):
    for k in (0,1):
        for a in model_graph.adj[n]:
            m=0
            if a in communities[k] and n in communities[k]:
                for b in model_graph.adj[a]:
                    if a in communities[k] and not(b in communities[k]):
                        m+=1
                if m<len(model_graph.adj[a]):
                    return True
    return False

In [5]:
def node_sets(communities,graph):
    boundary1=[]
    boundary2=[]
    for n in nx.nodes(model_graph):
        if cond1(n) and cond2(n):
            if n in communities[0]:
                boundary1.append(n)
            if n in communities[1]:
                boundary2.append(n)
    boundary=[boundary1,boundary2]
    internal=[list(communities[0]),list(communities[1])]
    for a in range(0,1):
        for n in communities[a]:
            if n in boundary[a]:
                internal[a].remove(n)
    return boundary, internal

In [6]:
def edges_sets(bound, internal, graph):
   
    eb=[]
    for n in range(len(bound[0])):
        sub_b=[]
        for neigh in graph.adj[bound[0][n]]:
            if neigh in bound[1]:
                sub_b.append([bound[0][n],neigh])
        eb.append(sub_b)

    for n in range(len(bound[1])):
        sub_b=[]
        for neigh in graph.adj[bound[1][n]]:
            if neigh in bound[0]:
                sub_b.append([bound[1][n],neigh])
        eb.append(sub_b)
    
    eint=[]
    for n in range(len(bound[0])):
        sub_int=[]
        for neigh in graph.adj[bound[0][n]]:
            if (neigh in internal[0]) or (neigh in internal[1]):
                sub_int.append([bound[0][n],neigh])  
        eint.append([sub_int])

    for n in range(len(bound[1])):
        sub_int=[]
        for neigh in graph.adj[bound[1][n]]:
            if (neigh in internal[0]) or (neigh in internal[1]):
                sub_int.append([bound[1][n],neigh])  
        eint.append([sub_int])
    return eb, eint

In [7]:
def p_value(boundary,e_bound,e_inter):
    mixed_bound=[]
    mixed_bound.extend(boundary[0])
    mixed_bound.extend(boundary[1])
    abs_b=len(mixed_bound)
    sum=0
    for n in range(len(mixed_bound)):
        sum=sum-0.5+len(e_inter[n])/(len(e_inter[n])+len(e_bound[n]))
    p_value=sum/abs_b
    return p_value

In [8]:
model_graph = nx.karate_club_graph()

communities=community.naive_greedy_modularity_communities(model_graph, resolution=0.5)

[boundary,internal]=node_sets(communities,model_graph)
[e_bound,e_inter]=edges_sets(boundary,internal,model_graph)
P=p_value(boundary,e_bound,e_inter)
print(P)


-0.0984848484848485


In [9]:
nt = Network(height="600px", width="100%", notebook=True, heading="Model graph")

### Next we populate the network using the graph we created.
nt.from_nx(model_graph)

angle_step = 2*math.pi/34
radius = 200

for node in nt.nodes:
    
    ### We get the id of the node and then compute its angle in the circular
    ### presentation
    idx = int(node['id'])
    angle = (idx-1)*angle_step
    
    ### Now we compute the x and y coordinates of the node.
    x = radius*math.cos(angle)
    y = radius*math.sin(angle)
    
    ### Finally, we set the properties of the node.
    node.update({'x':x, 'y':y, 'physics': False})

nt.show("Model_graph.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
